In [1]:
from bs4 import BeautifulSoup
from dateutil.parser import parse

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time

import sys

firefox_capabilities = DesiredCapabilities.FIREFOX
firefox_capabilities['marionette'] = True
firefox_capabilities['handleAlerts'] = True
firefox_capabilities['acceptSslCerts'] = True
firefox_capabilities['acceptInsecureCerts'] = True
geckoPath = 'driver/geckodriver.exe'

firefox = webdriver.Firefox(capabilities=firefox_capabilities, executable_path=geckoPath)
driver = webdriver.PhantomJS(executable_path='driver/phantomjs.exe')

try:
    firefox.get('https://www.crunchbase.com/organization/apple')
except:
    pass
driver.get('https://techcrunch.com/')
firefox.set_page_load_timeout(2)
driver.set_page_load_timeout(30)

def existence_in_crunchbase(name):
    base_url = 'www.crunchbase.com'
    url = 'https://techcrunch.com/search/'+name
    url_crunchbase = None
    try:
        driver.get(url)
    except KeyboardInterrupt:
        sys.exit()
    except:
        print('timeout of phantomjs')
        pass
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    blocks = soup.select("h2.post-title a")
    if len(blocks)>0:
        url_crunchbase = blocks[0]["href"]  
        name  = blocks[0].getText()
 
        if base_url not in url_crunchbase:
            url_crunchbase = None
    
    return url_crunchbase
            

def extract_company_techcrunch(name,url):
    try:
        firefox.get(url)
    except:
        pass
    founded = None
    employees = None
    company_name = ''
    blocks_dt = []
    blocks_dd = []
    try:
        soup = BeautifulSoup(firefox.page_source, "html.parser")
        blocks_dt = soup.select("div.details dt")
        blocks_dd = soup.select("div.details dd")
        company_name_tag = soup.select_one("#profile_header_heading")
        if company_name_tag == None:
            company_name = ''
        else:
            company_name = company_name_tag.getText()
    except:
        pass

    i = 0
    while company_name.lower() != name.lower() or len(blocks_dt)==0 :
        time.sleep(0.3)
        try:
            soup = BeautifulSoup(firefox.page_source, "html.parser")
            company_name_tag = soup.select_one("#profile_header_heading")
            if company_name_tag:
                company_name = company_name_tag.getText()
            else:
                company_name = ''
            blocks_dt = soup.select("div.details dt")
            blocks_dd = soup.select("div.details dd")
        except KeyboardInterrupt:
            sys.exit()
        except:
            pass
        i=i+1
        if i >= 100:
            print('fail to crawl ', name , ' in crunchbase')
            break

    
    if company_name.lower() == name.lower():
        for index,block in enumerate(blocks_dt):
            if 'Founded' in  block.getText() : 
                founded = blocks_dd[index].getText()
            if 'Employees' in block.getText():
                employees = blocks_dd[index].getText().split('|')[0]
    print( name , founded , employees)
    return founded,employees



In [2]:
import time
import json
import os

class CompaniesManager:
    def __init__(self):
        self.companies = []
        self.articles = []
        self.companies_name = []
        self.load_articles()
        self.load_companies()
        
    def load_articles(self):
        if os.path.isfile('data/raw_articles.json'):
            f = open('data/raw_articles.json')
            self.articles = json.load(f)
            f.close()
            
    def load_companies(self):
        if os.path.isfile('data/raw_companies.json'):
            f = open('data/raw_companies.json')
            self.companies = json.load(f)
            f.close()      
            for company in self.companies:
                name = company["name"]
                self.companies_name.append(name)
        
            
    def extract_companies(self):
        
        for index,article in enumerate(self.articles):
            for company_name in article["companies"]: 
                article_extraInfos = article["extra_infos"]
                for info in article_extraInfos:
                    if info["text"]== company_name:
                        relevance = info["relevance"]
                        count_in_article = info["count"]
                            
                if company_name in self.companies_name :
                    for company in self.companies:
                        if company_name == company["name"] and article["id"] not in company["articles"]: 
                            company["count"] = company["count"]+1
                            company["sentiment"]= company["sentiment"]+article["sentiment"]
                            company["articles"].append(article["id"])
                            extra_infos = {
                                 "id":article["id"] ,
                                 "count_in_article":count_in_article,
                                 "revelance": relevance       
                            }
                            company["extra_infos"].append(extra_infos)
                            
                else:
                    company = {
                        "name": company_name,
                        "dateFound": int(str(time.time()).split('.')[0]),
                        "count":1,
                        "sentiment": article["sentiment"],
                        "articles": [article["id"]],
                        "extra_infos":
                        [
                            {
                             "id": article["id"],
                             "count_in_article":count_in_article,
                             "revelance": relevance
                            }  
                        ]    
                    }
                    self.companies.append(company)
                    self.companies_name.append(company_name)
            
        self.save_to_disk()
        
    def extend_crunch(self):
        
        i = 0
        for index,company in enumerate(self.companies):
            name = company["name"]
            if "search_label" not in company:
                url_crunchbase = existence_in_crunchbase(name)
                
                if url_crunchbase != None:
                    print(name, ' in crunchbase')
                    i=i+1
                    founded,employees = extract_company_techcrunch(name,url_crunchbase)
                    company["search_label"] = url_crunchbase
                    if founded is not None:
                        company["foundationDate"] = founded
                    if employees is not None:
                        company["number_of_employees"] = employees
                    
                else:
                    company["search_label"] = str(0)
                    print(name, 'not in crunchbase')
            else:
                print("skip: ", name)
            
            if (index+1)%20 == 0:
                self.save_to_disk()
        self.save_to_disk()            
                                
    def save_to_disk(self):
        with open('data/raw_companies.json', 'w') as company_file:
            json.dump(self.companies, company_file,indent = 2)
            company_file.close()
                
                
                
            
    

In [4]:
CM = CompaniesManager()
CM.extract_companies()
try:
    CM.extend_crunch()
except KeyboardInterrupt:
    sys.exit()

skip:  SENSORO
skip:  Microsoft Corp
skip:  Sumitomo
skip:  Manchester City Verve
skip:  SalesWings
skip:  SA tech
skip:  Avito
skip:  Tshimologong Innovation Hub
skip:  Flickr
skip:  Niantic Labs
skip:  Oblong
skip:  Osterhout Design Group
skip:  Niantic
skip:  Oblong Industries
skip:  Mezzanine
skip:  The Pokemon Company
skip:  GamesBeat
skip:  BNJ Marketing
skip:  DormaKaba
skip:  Berkshire companies
skip:  Bezos
skip:  Private Equity
skip:  Utrip
skip:  Greylock
skip:  Elsight
skip:  LiveU
skip:  WAN
skip:  Vodaphone
skip:  Optus
skip:  GoRemote
skip:  Agri Alliance
skip:  Bitbond
skip:  Obotritia
skip:  FiveAI
skip:  GuestReady
skip:  Carspring
skip:  Moneybox
skip:  Drivetribe
skip:  Realla
skip:  Habito
skip:  Sling & Stone
skip:  Crowdcube
skip:  Cali Rice
skip:  Nominet Trust
skip:  3D Robotics
skip:  AgFunder
skip:  Airware
skip:  Arbe Robotics
skip:  Zipline International
skip:  Redbird
skip:  Bloomberg Businessweek
skip:  Yuneec Electric Aviation
skip:  CyPhy Works
skip:  E

Emerge Analytics not in crunchbase
Luno  in crunchbase
Luno 2013 11 - 50 
Lula Lend not in crunchbase
Isazi Consulting not in crunchbase
AlphaCode not in crunchbase
NCR  in crunchbase
NCR 1884 10k + 
Seedstars World  in crunchbase
Seedstars World None None
Fomo Travel not in crunchbase
Brightcap not in crunchbase
Johannesburg Social Network not in crunchbase
GD Industries not in crunchbase
Medium-scale Enterprises not in crunchbase
m.Labs not in crunchbase
Shosholoza Meyl not in crunchbase
SiMODiSA not in crunchbase
Barclays Africa Group not in crunchbase
Landmark Barclays Africa not in crunchbase
Barclay not in crunchbase
Jumo  in crunchbase
Jumo February 1, 2010 1 - 10 
PricewaterhouseCoopers not in crunchbase
MoneyTree  in crunchbase
MoneyTree April 23, 2012 11 - 50 
Rebright Partners  in crunchbase
Rebright Partners January 11, 2008 None
Archimedes Labs  in crunchbase
Archimedes Labs May 1, 2005 None
Edgars not in crunchbase
SA Ignite  in crunchbase
SA Ignite 2006 11 - 50 
TRECC no

Citymapper  in crunchbase
Citymapper 2011 11 - 50 
Microsemi  in crunchbase
Microsemi 1960 5k - 10k 
Altera  in crunchbase
Altera 1983 5 in Crunchbase
Cadence Design Systems  in crunchbase
Cadence Design Systems 1988 5k - 10k 
United Design Systems not in crunchbase
Linley Group not in crunchbase
Synopsys  in crunchbase
Synopsys 1986 5k - 10k 
Atmel  in crunchbase
Atmel January 1, 1984 5k - 10k 
HP Enterprise not in crunchbase
Juniper  in crunchbase
Juniper 1996 51 - 100 
livestreaming not in crunchbase
Deezer  in crunchbase
Deezer August 1, 2006 501 - 1k 
Ultra  in crunchbase
Ultra May 1, 2014 1 - 10 
Goldenvoice  in crunchbase
Goldenvoice 1981 11 - 50 
fiat not in crunchbase
PST Consulting not in crunchbase
TGPR  in crunchbase
TGPR Unknown 1 - 10 
NOLs not in crunchbase
JumpStart Inc.  in crunchbase
JumpStart Inc. 2003 None
Fog Creek Software  in crunchbase
Fog Creek Software September 1, 2000 11 - 50 
Allied Gear not in crunchbase
Internet Corporation not in crunchbase
Continuum Ana

Blue Origin  in crunchbase
Blue Origin 2011 251 - 500 
Peddler Brewing Company not in crunchbase
New Shephard not in crunchbase
TechBargains  in crunchbase
TechBargains 1999 501 - 1k 
HostGator  in crunchbase
HostGator 2002 3 in Crunchbase
PCMag  in crunchbase
PCMag 1982 2 in Crunchbase
AMD  in crunchbase
AMD May 1, 1969 10k + 
Vizio  in crunchbase
Vizio 2002 251 - 500 
DellDell not in crunchbase
Wordpress  in crunchbase
fail to crawl  Wordpress  in crunchbase
Wordpress None None
CloudFlare  in crunchbase
CloudFlare July 1, 2009 251 - 500 
Heinz  in crunchbase
Heinz 1869 10k + 
Sourcify  in crunchbase
Sourcify 2016 None found in Crunchbase
Digitalux not in crunchbase
CNBC  in crunchbase
CNBC April 17, 1989 33 in Crunchbase
TINYpulse  in crunchbase
TINYpulse November 1, 2012 4 in Crunchbase
CareerBuilder  in crunchbase
CareerBuilder 1995 5k - 10k 
Veriato  in crunchbase
Veriato 1998 51 - 100 
Come Recommended  in crunchbase
Come Recommended December 1, 2008 11 - 50 
Red Beach Advisors  

Floodgate  in crunchbase
Floodgate 2006 None
Compte Nickel  in crunchbase
Compte Nickel 2014 11 - 50 
Duo Security  in crunchbase
Duo Security 2010 251 - 500 
Tala not in crunchbase
Duo  in crunchbase
Duo 1996 11 - 50 
83North  in crunchbase
83North June 30, 2006 None
Greylock IL not in crunchbase
Wonga  in crunchbase
Wonga 2006 501 - 1k 
Hybris  in crunchbase
Hybris August 11, 1997 501 - 1k 
Take-Two not in crunchbase
IronSource  in crunchbase
IronSource 2010 19 in Crunchbase
ScaleIO  in crunchbase
ScaleIO 2011 11 - 50 
SocialPoint  in crunchbase
SocialPoint 2012 1 - 10 
Farmers Edge not in crunchbase
Sightline not in crunchbase
Sightline Innovation not in crunchbase
Kleiner Perkins not in crunchbase
Manitoba Technology Accelerator not in crunchbase
BL Photonics not in crunchbase
Wolf Trax  in crunchbase
Wolf Trax 2000 1 - 10 
DuPont  in crunchbase
DuPont July 19, 1802 50 in Crunchbase
TC  in crunchbase
fail to crawl  TC  in crunchbase
TC None None
Future Labs not in crunchbase
French

...

Simbe Robotics not in crunchbase
ReActive Robotics GmbH not in crunchbase
Ascent Robotics not in crunchbase
Cafe X Technologies not in crunchbase
AIO Robotics not in crunchbase
Aarav Unmanned Systems not in crunchbase
UBTech not in crunchbase
RooBo not in crunchbase
Matternet not in crunchbase
Kimera Systems not in crunchbase
Raptor Maps not in crunchbase
Superflex not in crunchbase
Jane Technologies not in crunchbase
gWomen not in crunchbase
Volusion not in crunchbase
Infiniscene not in crunchbase
Instinet not in crunchbase
BusinessCollective not in crunchbase
CHASE not in crunchbase
JUGGLE not in crunchbase
Traackr not in crunchbase
BuzzSumo not in crunchbase
Followerwonk not in crunchbase
Moz not in crunchbase
Logicalis Healthcare Solutions not in crunchbase
ezLandlordForms not in crunchbase
Vedsutra not in crunchbase
European Circuits not in crunchbase
Pando not in crunchbase
LeapLearner not in crunchbase
LeapLearner China not in crunchbase
Expert Market not in crunchbase
LearnLaun

Kwippit not in crunchbase
MTV Networks not in crunchbase
Authentic Brands Group not in crunchbase
Abartys Health not in crunchbase
Jukedeck not in crunchbase
SkinVision USA not in crunchbase
Autism Spectrum Australia not in crunchbase
LiveCode not in crunchbase
Richett Media not in crunchbase
McAfee not in crunchbase
Kaspersky Lab not in crunchbase
Wallarm not in crunchbase
Columbus Nova Technology Partners not in crunchbase
Cars.com not in crunchbase
MedRhythms not in crunchbase
Putney Inc. not in crunchbase
Safeguard not in crunchbase
Dechra Pharmaceuticals PLC not in crunchbase
Dechra Holdings US Inc. not in crunchbase
Safeguard Scientifics Inc. not in crunchbase
Newport Strategies Inc. not in crunchbase
Rippleshot not in crunchbase
Zulily not in crunchbase
Firstbird not in crunchbase
Evan Thompson & Associates not in crunchbase
Social Nickel not in crunchbase
JustReachOut.io not in crunchbase
MarketMuse not in crunchbase
E-Man Labs not in crunchbase
BMB not in crunchbase
MailChimp 

RadPad Canva not in crunchbase
Pixabay not in crunchbase
CarAdvise not in crunchbase
Comnplus not in crunchbase
qTest Manager not in crunchbase
SEED SPOT not in crunchbase
Booz Allen Hamilton not in crunchbase
Vairdo not in crunchbase
Dronesmith not in crunchbase
Audicus not in crunchbase
Girlboss Radio not in crunchbase
Bizdaq not in crunchbase
Spora not in crunchbase
FE International not in crunchbase
Ebates not in crunchbase
Honest Beef not in crunchbase
GSX Solutions not in crunchbase
Honest Beef Company not in crunchbase
Actifio not in crunchbase
Soom Planning not in crunchbase
Silicon Harlem not in crunchbase
SproutSocial not in crunchbase
ShareShare not in crunchbase
Edge Tech Labs not in crunchbase
crowdsourcing not in crunchbase
person/company not in crunchbase
Easy Solutions not in crunchbase
EcoTech Visions not in crunchbase
Voxer not in crunchbase
Rocket Fiber not in crunchbase
ROHO not in crunchbase
Marvel Entertainment not in crunchbase
Mortgaged Backed Securities not in 

Lehman Brothers not in crunchbase
Kompli Global not in crunchbase
ex-Goldman Sachs not in crunchbase
DueDil not in crunchbase
Suade not in crunchbase
Kompli not in crunchbase
Aosphere not in crunchbase
Wild Flowers not in crunchbase
Splittable not in crunchbase
ManoMano not in crunchbase
Captify not in crunchbase
Reach Robotics not in crunchbase
Cambridge Medical Robotics not in crunchbase
startup Automata not in crunchbase
Moley Robotics not in crunchbase
Consequential Robotics not in crunchbase
MiRo not in crunchbase
RAAS not in crunchbase
Open Bionics not in crunchbase
Innovate UK not in crunchbase
Oxford Technology not in crunchbase
Primo Toys not in crunchbase
Emotech not in crunchbase
Constant Contact not in crunchbase
SendBlaster not in crunchbase
Sendinblue not in crunchbase
VirticalResponse not in crunchbase
Mailjet not in crunchbase
ReachMail not in crunchbase
VerticalResponse not in crunchbase
Satchel RNF Digital Innovation not in crunchbase
RNF Digital Innovation not in cru

HSDC not in crunchbase
Verifone not in crunchbase
Hewlett Packard not in crunchbase
Abercrombie not in crunchbase
Straight Path Communications not in crunchbase
airwave not in crunchbase
Charter not in crunchbase
Betaworks not in crunchbase
La French Tech not in crunchbase
French telecom not in crunchbase
medtech not in crunchbase
Microsoft Research not in crunchbase
Bing.com not in crunchbase
FUSE Labs not in crunchbase
self-driving not in crunchbase
4Front Logistics not in crunchbase
DOT not in crunchbase
Stephens Inc not in crunchbase
AI technologies not in crunchbase
ClearMetal not in crunchbase
Signifyd not in crunchbase
Prevedere not in crunchbase
Illusive Networks not in crunchbase
Infocyte not in crunchbase
MindMeld not in crunchbase
USAA not in crunchbase
Uniqlo not in crunchbase
IDG not in crunchbase
MapD not in crunchbase
Nectar Labs not in crunchbase
Dosa Restaurant not in crunchbase
Chegg not in crunchbase
Conde Nast not in crunchbase
Genwi not in crunchbase
Xamarin not in